In [5]:
import os
import cv2
import argparse
import pandas as pd
import numpy as np

from tqdm import tqdm

# parser = argparse.ArgumentParser(description='')
# parser.add_argument('--f', required=False, default='submission.csv')
# args = parser.parse_args()

root_dir = '/opt/ml/input/data/'
save_path = '/opt/ml/code/mmdetection_trash/submission_image2'
os.makedirs(save_path, exist_ok=True)

# df = pd.read_csv(f'/opt/ml/code/mmdetection_trash/{args.f}')
d1 = 'detectors_x101_64x4d_adam2'
d2 = 'detectors_x101_64x4d_adam2_epoch18'
df = pd.read_csv(f'/opt/ml/code/mmdetection_trash/work_dirs/{d2}.csv')
# df = pd.read_csv(f'/opt/ml/code/mmdetection_trash/en4.csv')
image_id = df.image_id
prediction_string = df.PredictionString

classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

COLORS = [
    (39, 129, 113), 
    (164, 80, 133), 
    (83, 122, 114), 
    (99, 81, 172), 
    (95, 56, 104), 
    (37, 84, 86), 
    (14, 89, 122),
    (80, 7, 65), 
    (10, 102, 25), 
    (90, 185, 109),
    (106, 110, 132)
]

for i in tqdm(range(837)):    
    img_path = os.path.join(root_dir, image_id[i])
    # print(img_path)
    img = cv2.imread(img_path)
    annot = prediction_string[i].split(' ')
    annot = [annot[i * 6:(i + 1) * 6] for i in range((len(annot) + 6 - 1) // 6 )] 
    annot.remove([''])
    output_img = img.copy()
    for bbox in annot:
        if 'nan' in bbox:
            print(image_id[i])
            continue
        bbox = list(map(float, bbox))
        bbox = np.array(bbox).astype(int)
        label = int(bbox[0])
        xmin, ymin, xmax, ymax = bbox[2:6]

        color = COLORS[label]
        cv2.rectangle(output_img, (xmin, ymin), (xmax, ymax), color, 2)        
        text_size = cv2.getTextSize(classes[label], cv2.FONT_HERSHEY_PLAIN, 1, 1)[0]
        cv2.rectangle(output_img, (xmin, ymin), (xmin + text_size[0] + 2, ymin + text_size[1] + 6), color, -1)
        cv2.putText(
            output_img, classes[label],
            (xmin, ymin + text_size[1] + 4), cv2.FONT_ITALIC, 0.5,
            (255, 255, 255), 1, cv2.LINE_AA)        
    result_img = np.concatenate((img, output_img), axis=1)
    
    cv2.imwrite(os.path.join(save_path, image_id[i].replace('/', '-')), result_img)



100%|██████████| 837/837 [00:14<00:00, 57.56it/s]


In [ ]:
model = dict()
model['resnet50'] = dict(pretrained='torchvision://resnet50',
                         backbone=dict(type='ResNet',
                                       depth=50,
                                       num_stages=4,
                                       out_indices=(0, 1, 2, 3),
                                       frozen_stages=1,
                                       norm_cfg=dict(type='BN', requires_grad=True),
                                       norm_eval=True,
                                       style='pytorch'))
model['resnset50_caffe'] = dict(pretrained='open-mmlab://detectron2/resnet50_caffe',
                                backbone=dict(type='ResNet',
                                              depth=50,
                                              num_stages=3,
                                              strides=(1, 2, 2),
                                              dilations=(1, 1, 1),
                                              out_indices=(2, ),
                                              frozen_stages=1,
                                              norm_cfg=norm_cfg,
                                              norm_eval=True,
                                              style='caffe'),
                
model['resnet101_caffe'] = dict(pretrained = 'open-mmlab://detectron2/resnet101_caffe',
                                backbone = dict(depth=101))
model['resnet101'] = dict(pretrained='torchvision://resnet101',
                          backbone=dict(depth=101))
model['resnet50_caffe'] = dict(pretrained='open-mmlab://detectron2/resnet50_caffe',
                               backbone=dict(norm_cfg=dict(requires_grad=False), norm_eval=True, style='caffe'))
model['resnet50_caffe'] = dict(pretrained='open-mmlab://detectron2/resnet50_caffe',
                                backbone=dict(norm_cfg=dict(requires_grad=False), norm_eval=True, style='caffe'),
                                roi_head=dict(bbox_head=dict(num_classes=1)))
model = dict(roi_head=dict(bbox_head=dict(reg_decoded_bbox=True,
                                          loss_bbox=dict(type='BoundedIoULoss', loss_weight=10.0))))
model['resnext101'] = dict(pretrained='open-mmlab://resnext101_32x4d',
                        backbone=dict(type='ResNeXt',
                                      depth=101,
                                      groups=32,
                                      base_width=4,
                                      num_stages=4,
                                      out_indices=(0, 1, 2, 3),
                                      frozen_stages=1,
                                      norm_cfg=dict(type='BN', requires_grad=True),
                                      style='pytorch'))